Stage 3: SQL Analytics (DuckDB)
 
**Description:** In this stage, we simulate a modern **Data Lakehouse** architecture. Instead of loading data into a traditional heavy database (like PostgreSQL), we use **DuckDB** — an in-process OLAP SQL engine. This allows us to run complex SQL queries directly on our optimized `Parquet` files.

**Business Objectives:**
1. **Market Segmentation:** Analyze average pricing and nutrient density across the ML-generated clusters.
2. **Value Analysis:** Identify the top 3 "Best Value" products in each category using Window Functions.
3. **Inventory Risk:** Detect premium products with critical stock levels to generate alerts for the Supply Chain team.

**Tech Stack:** `DuckDB`, `SQL`, `Pandas`

In [4]:
import duckdb
import pandas as pd

# --- Configuration ---
# Connecting to the "Gold Layer" (Cleaned & Enriched Data)
INPUT_FILE = '../data/processed/supplements_enriched.parquet'

print("Connecting to Data Lakehouse via DuckDB...")

# Initialize in-memory database connection
con = duckdb.connect()

# ==============================================================================
# REPORT 1: Market Segmentation Overview
# Business Question: "How does pricing and potency differ across our customer segments?"
# ==============================================================================
print("\n--- [Report 1] Market Segmentation Overview ---")

query_overview = f"""
SELECT 
    segment_name,
    COUNT(*) as total_products,
    ROUND(AVG(price_eur), 2) as avg_price_eur,
    ROUND(AVG(vitamin_c_mg), 0) as avg_vit_c_mg,
    ROUND(AVG(magnesium_mg), 0) as avg_magnesium_mg
FROM '{INPUT_FILE}'
GROUP BY segment_name
ORDER BY avg_price_eur DESC
"""

df_overview = con.execute(query_overview).df()
display(df_overview)

# ==============================================================================
# REPORT 2: Best Value Products (Top 3 per Category)
# Business Question: "Which products offer the most active ingredients per Euro?"
# Technical Skill: Window Functions (ROW_NUMBER)
# ==============================================================================
print("\n--- [Report 2] Top 3 'Best Value' Products per Category ---")

query_ranking = f"""
WITH RankedProducts AS (
    SELECT 
        product_name,
        category,
        price_eur,
        price_per_1000mg,
        -- Partition by Category to restart ranking for each product type
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY price_per_1000mg ASC) as rank_id
    FROM '{INPUT_FILE}'
)
SELECT 
    category,
    product_name,
    price_eur,
    ROUND(price_per_1000mg, 2) as eur_per_1000mg_active
FROM RankedProducts
WHERE rank_id <= 3
ORDER BY category, eur_per_1000mg_active
"""

df_ranking = con.execute(query_ranking).df()
display(df_ranking)

# ==============================================================================
# REPORT 3: Inventory Risk Alert
# Business Question: "Which Premium products are running low on stock?"
# ==============================================================================
print("\n--- [Report 3] Low Stock Alert (Premium Segment) ---")

query_stock = f"""
SELECT 
    product_name,
    stock_level,
    price_eur,
    last_updated
FROM '{INPUT_FILE}'
WHERE segment_name LIKE '%Premium%' 
  AND stock_level < 20
ORDER BY stock_level ASC
LIMIT 10
"""

df_stock = con.execute(query_stock).df()
display(df_stock)

Connecting to Data Lakehouse via DuckDB...

--- [Report 1] Market Segmentation Overview ---


,segment_name,total_products,avg_price_eur,avg_vit_c_mg,avg_magnesium_mg
0,High-Potency Premium,1381,40.23,56.0,76.0
1,Balanced / Standard,316,39.87,798.0,50.0
2,Budget Essentials,303,39.12,40.0,350.0



--- [Report 2] Top 3 'Best Value' Products per Category ---


,category,product_name,price_eur,eur_per_1000mg_active
0,Magnesium,NutriLife Magnesium Citrate Basic,11.57,25.48
1,Magnesium,NutriLife Magnesium Citrate Basic,11.97,26.02
2,Magnesium,NutriLife Night Relax Magnesium Basic,11.30,28.18
3,Multivitamin,NutriLife A-Z Complete Basic,10.23,39.35
4,Multivitamin,NutriLife A-Z Complete Basic,11.36,41.46
5,Multivitamin,NutriLife Men's Multi Power Basic,12.28,43.86
6,Omega-3,NutriLife Heart Health Omega Basic,11.55,87.50
7,Omega-3,NutriLife Heart Health Omega Basic,12.33,103.61
8,Omega-3,NutriLife Deep Sea Omega Basic,11.04,105.14
9,Vitamin C,NutriLife Immune C Boost Basic,10.09,11.58



--- [Report 3] Low Stock Alert (Premium Segment) ---


,product_name,stock_level,price_eur,last_updated
0,NutriLife Vitamin D3 + K2 Basic,0,12.94,2025-12-13 02:06:50.008753
1,NutriLife Zinc Bisglycinate Pro,0,34.89,2025-12-18 02:06:50.010031
2,NutriLife Krill Oil Gold Ultra,1,54.52,2025-12-14 02:06:50.007391
3,NutriLife Krill Oil Gold Pro,1,37.55,2025-12-25 02:06:50.008570
4,NutriLife D3 Max Basic,2,35.83,2025-12-06 02:06:50.010096
5,NutriLife Heart Health Omega Pro,2,31.52,2025-12-29 02:06:50.010203
6,NutriLife A-Z Complete Basic,3,28.17,2025-12-17 02:06:50.007561
7,NutriLife Deep Sea Omega Ultra,3,41.97,2025-12-11 02:06:50.008996
8,NutriLife D3 Max Pro,4,25.74,2026-01-01 02:06:50.008319
9,NutriLife Family Multi Basic,4,38.01,2025-12-29 02:06:50.009879
